In [1]:
# First import OpenCV, NumPY and MatPlotLib as we will use these libraries
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import * 
%matplotlib inline


In [2]:
# To generate a vector of avg values

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import scipy.cluster.vq as scv
import os
    
def colormap2arr(arr,cmap):    
    # http://stackoverflow.com/questions/3720840/how-to-reverse-color-map-image-to-scalar-values/3722674#3722674
    gradient=cmap(np.linspace(0.0,1.0,100))

    # Reshape arr to something like (240*240, 4), all the 4-tuples in a long list...
    arr2=arr.reshape((arr.shape[0]*arr.shape[1],arr.shape[2]))

    # Use vector quantization to shift the values in arr2 to the nearest point in
    # the code book (gradient).
    code,dist=scv.vq(arr2,gradient)

    # code is an array of length arr2 (240*240), holding the code book index for
    # each observation. (arr2 are the "observations".)
    # Scale the values so they are from 0 to 1.
    values=code.astype('float')/gradient.shape[0]

    # Reshape values back to (240,240)
    values=values.reshape(arr.shape[0],arr.shape[1])
    values=values[::-1]
    return values

In [3]:
def image2Vector(imgname):
    arr=plt.imread(imgname)
    imgWidth = arr.shape[0]
    imgHeight = arr.shape[1]

    #print(imgWidth, imgHeight)

    leftPart=arr[:,int(np.around(imgWidth/2)):(imgWidth-1),:]

    #print(leftPart.shape)

    values=colormap2arr(leftPart,cm.jet)    

    #print(values.shape)

    avgValues=np.mean(values, axis=1, dtype=np.float32)

    #print(avgValues.shape)

    # Proof that it works:
    #plt.imshow(values,interpolation='bilinear', cmap=cm.jet,
    #       origin='lower', extent=[-3,3,-3,3])
    #plt.show()
    
    #print(avgValues)
    
    return avgValues

#image2Vector("1000_c.png")

In [4]:
def image2Vector_Log(imgname):
    arr=plt.imread(imgname)
    imgWidth = arr.shape[0]
    imgHeight = arr.shape[1]

    #print(imgWidth, imgHeight)

    leftPart=arr[:,int(np.around(imgWidth/2)):(imgWidth-1),:]

    values=colormap2arr(leftPart,cm.jet)    

    #avgValues=np.mean(values, axis=1, dtype=np.float32)

    avgValues=np.log10(np.mean(np.power(10,values), axis=1, dtype=np.float32))
    
    return avgValues


In [ ]:
a=np.array([1,2,3,4,5])
b=a[3:5]
print(b)

In [4]:
def image2Vector_2d(imgname):
    arr=plt.imread(imgname)
    imgWidth = arr.shape[0]
    imgHeight = arr.shape[1]

    #print(imgWidth, imgHeight)

    leftPart=arr[:,(imgWidth-64):imgWidth,:]

    values=colormap2arr(leftPart,cm.jet)    

    return Values.flatten()

In [5]:
#!/usr/bin/python
def create_pairs_2d(datasource,foldername):
    id_label_arr=np.array(datasource)

    #rows=np.ndarray(shape=(2*imgHeight+1), dtype = np.float)
    rows=np.zeros((2*imgHeight+1,), dtype = np.float)

    first=True

    #print(rows)
    #print("*****************")

    for pair in id_label_arr:

        current_file = os.path.join(".", foldername,str(pair[0])+"_c.png")
        voltage_file= os.path.join(".", foldername,str(pair[0])+"_v.png")
    
        #print(current_file)
        if os.path.exists(current_file) and os.path.exists(voltage_file):
            # For submission 1-6
            #current_array=image2Vector(current_file)
            #voltage_array=image2Vector(voltage_file)
            
            # For submission after 6. We hope to improve the prepared data
            current_array=image2Vector_Log(current_file)
            voltage_array=image2Vector_Log(voltage_file)
                       
            row=np.hstack((current_array,voltage_array,[pair[1]]))
            #print(row.shape)
        
            if first:
                print(pair)            
                print(current_file)
                print(current_array)
                rows=row
                print(row)
                first=False
                #break
            else:
                rows=np.concatenate((rows,row), axis=0)
        else:
            print("Missed:"+current_file+", or "+ voltage_file)
        #print(rows.shape)
        #print(rows)

    newRows=rows.reshape(int(rows.shape[0]/257),257)
        #print(newRows.shape)
    return newRows

In [6]:
# Save the data files into a format compatible with CNTK text reader
def save_as_txt(filename, ndarray):
    dir = os.path.dirname(filename)

    if not os.path.exists(dir):
        os.makedirs(dir)

    first=True    
    if not os.path.isfile(filename):
        print("Saving", filename )
        with open(filename, 'w') as f:
            labels = list(map(' '.join, np.eye(11, dtype=np.uint).astype(str))) #11 is the num of kinds we will classify into
            
            for row in ndarray:
                row_str = row.astype(str)
                #print(row[-1])
                #print(type(row[-1]))
                label=np.int(np.round(float(row[-1]),0))
                label_str = labels[label]
                feature_str = ' '.join(row_str[:-1])
                f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                
                if first:
                    #print('|labels {} |features {}\n'.format(label_str, feature_str))
                    #print(label)
                    #print(label_str)
                    first=False
                    #break
    else:
        print("File already exists", filename)

In [7]:
import csv

with open('train_labels.csv', newline='') as csvfile:
    id_labels = list(csv.reader(csvfile))

#print(id_labels)

with open('submission_format.csv', newline='') as csvfile:
    id_labels_test = list(csv.reader(csvfile))

#print(id_labels_test)



In [8]:
    
imgHeight=128

# Save the train and test files (prefer our default path for the data)
data_dir = os.path.join(".", "PLAID")

train_pairs=create_pairs(id_labels,"train") 
print ('Writing train text file...')
save_as_txt(os.path.join(data_dir, "train_2d.txt"), train_pairs)  #Before submission 7, "train" instead

test_pairs=create_pairs(id_labels_test,"test") 
print ('Writing test text file...')
save_as_txt(os.path.join(data_dir, "test_2d.txt"), test_pairs) #Before submission 7, "test" instead

print('Done')

Missed:./train/id_c.png, or ./train/id_v.png
['1000' '8']
./train/1000_c.png
[5.5116707e-01 6.5912443e-01 6.0018742e-01 2.7016672e-01 2.6991546e-01
 4.5745564e-01 4.5000011e-01 2.8639387e-02 2.1715457e-02 4.2999989e-01
 4.3981683e-01 2.4240634e-01 2.5517372e-02 2.5429639e-01 4.2999989e-01
 2.4946339e-01 1.4406181e-02 7.5630158e-02 2.6439300e-01 2.4981697e-01
 2.8009736e-03 2.9870404e-03 1.5272550e-02 1.5579637e-02 1.4960625e-03
 7.4864965e-04 6.1384491e-03 1.0000011e-02 1.8727049e-04 3.7451199e-04
 1.8727049e-04 1.8727049e-04 1.8727049e-04 1.3093557e-03 1.8727049e-04
 7.4864965e-04 5.6162110e-04 3.7451199e-04 1.8727049e-04 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.8727049e-04 1.1225168e-03 1.8727049e-04
 0.0000000e+00 0.0000000e+00 0.0000000e+00 5.6162110e-04 0.0000000e+00
 0.0000000e+00 1.8727049e-04 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.8727049e-04 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00


Writing test text file...
Saving ./PLAID/test_log.txt
Done


In [9]:
2964 /257

11.53307392996109

In [10]:
list(map(' '.join, np.eye(11, dtype=np.uint).astype(str))) 

['1 0 0 0 0 0 0 0 0 0 0',
 '0 1 0 0 0 0 0 0 0 0 0',
 '0 0 1 0 0 0 0 0 0 0 0',
 '0 0 0 1 0 0 0 0 0 0 0',
 '0 0 0 0 1 0 0 0 0 0 0',
 '0 0 0 0 0 1 0 0 0 0 0',
 '0 0 0 0 0 0 1 0 0 0 0',
 '0 0 0 0 0 0 0 1 0 0 0',
 '0 0 0 0 0 0 0 0 1 0 0',
 '0 0 0 0 0 0 0 0 0 1 0',
 '0 0 0 0 0 0 0 0 0 0 1']

In [11]:
np.log10(1110)

3.0453229787866576

In [12]:
import numpy as np
v=[1,2,3]
va=np.array(v)
np.log10(np.mean(np.power(10,v)))

2.568201724066995